In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from tabulate import tabulate

# 1. Carga de datos
base_path = "Proyectos_Datos_Edicion_01_EVM"
df = pd.read_csv(f"{base_path}/data/historico_real.csv")

# 2. Procesamiento de Indicadores Mensuales
df_mensual = df.groupby("Mes").agg({"PV":"sum", "EV":"sum", "AC":"sum"}).reset_index()

# 3. Filtrado y Cálculos Mes 8
df_m8 = df[df["Mes"] == 8].copy()
df_m8["CPI"] = (df_m8["EV"] / df_m8["AC"]).round(2)
df_m8["SPI"] = (df_m8["EV"] / df_m8["PV"]).round(2)

# Cálculo de Totales para la Tabla
totales = pd.DataFrame({
    "Actividad": ["TOTAL"],
    "BAC": [df_m8["BAC"].sum()],
    "PV": [df_m8["PV"].sum()],
    "EV": [df_m8["EV"].sum()],
    "AC": [df_m8["AC"].sum()],
    "CPI": [(df_m8["EV"].sum() / df_m8["AC"].sum())],
    "SPI": [(df_m8["EV"].sum() / df_m8["PV"].sum())]
})

# 4. Configuración Visual Estilo Dark Corporativo
plt.style.use("dark_background")
fig = plt.figure(figsize=(20, 10), facecolor="#0b1c2d")

# --- A. CURVA S INTEGRAL CON ETIQUETAS ---
ax1 = fig.add_subplot(1, 2, 1)
ax1.set_facecolor("#0b1c2d")

# Líneas de tendencia
ax1.plot(df_mensual["Mes"], df_mensual["PV"], color="#3498db", label="PV (Planeado)", lw=2.5, alpha=0.8)
ax1.plot(df_mensual[df_mensual["Mes"]<=8]["Mes"], df_mensual[df_mensual["Mes"]<=8]["EV"], color="#2ecc71", label="EV (Real)", marker='o', lw=3)
ax1.plot(df_mensual[df_mensual["Mes"]<=8]["Mes"], df_mensual[df_mensual["Mes"]<=8]["AC"], color="#e74c3c", label="AC (Costo)", ls="--", lw=2)

# Etiquetado de datos específicos Mes 8
m8_pv = df_mensual.loc[df_mensual["Mes"]==8, "PV"].values[0]
m8_ev = df_mensual.loc[df_mensual["Mes"]==8, "EV"].values[0]
m8_ac = df_mensual.loc[df_mensual["Mes"]==8, "AC"].values[0]

ax1.annotate(f'${m8_pv:,.0f}', (8, m8_pv), textcoords="offset points", xytext=(0,10), ha='center', color="#3498db", fontsize=9, fontweight='bold')
ax1.annotate(f'${m8_ev:,.0f}', (8, m8_ev), textcoords="offset points", xytext=(0,-15), ha='center', color="#2ecc71", fontsize=9, fontweight='bold')
ax1.annotate(f'${m8_ac:,.0f}', (8, m8_ac), textcoords="offset points", xytext=(0,10), ha='center', color="#e74c3c", fontsize=9, fontweight='bold')

ax1.set_title("Gobernanza de Proyectos: Evolución Curva S Integral", fontsize=15, pad=20)
ax1.yaxis.set_major_formatter(ticker.StrMethodFormatter('${x:,.0f}'))
ax1.set_xlabel("Meses de Ejecución")
ax1.legend(loc="upper left")
ax1.grid(True, alpha=0.1)

# --- B. ANÁLISIS DE CUADRANTES CON ETIQUETAS DE ACTIVIDAD ---
ax2 = fig.add_subplot(1, 2, 2)
ax2.set_facecolor("#0b1c2d")

# Gráfico de burbujas
scatter = ax2.scatter(df_m8["SPI"], df_m8["CPI"], s=df_m8["BAC"]/1000,
                      c=df_m8["CPI"], cmap="RdYlGn", alpha=0.7, edgecolors="white", linewidth=1)

# Etiquetar cada burbuja con el nombre de la actividad
for i, row in df_m8.iterrows():
    ax2.text(row["SPI"], row["CPI"] + 0.015, row["Actividad"], fontsize=8, ha='center', color="white", alpha=0.9)

# Líneas de umbral (1.0)
ax2.axhline(1, color="white", lw=1, ls="--", alpha=0.5)
ax2.axvline(1, color="white", lw=1, ls="--", alpha=0.5)

ax2.set_xlabel("Eficiencia Tiempo (SPI)")
ax2.set_ylabel("Eficiencia Costo (CPI)")
ax2.set_title("Diagnóstico de Cuadrantes (Burbuja = Presupuesto)", fontsize=15, pad=20)
ax2.set_xlim(0.6, 1.2)
ax2.set_ylim(0.6, 1.2)

plt.tight_layout()
plt.savefig(f"{base_path}/dashboards/analisis_integral_mes8_detallado.png", dpi=300)
plt.show()

# --- 5. REPORTE DE EJECUCIÓN MES 8 (CON TOTALES) ---
df_final_table = pd.concat([df_m8, totales], ignore_index=True)
df_final_table = df_final_table[["Actividad", "BAC", "PV", "EV", "AC", "CPI", "SPI"]]

# Formateo de moneda (Sin decimales)
for col in ["BAC", "PV", "EV", "AC"]:
    df_final_table[col] = df_final_table[col].apply(lambda x: f"${x:,.0f}")

# Formateo de índices
df_final_table["CPI"] = df_final_table["CPI"].apply(lambda x: f"{x:.2f}")
df_final_table["SPI"] = df_final_table["SPI"].apply(lambda x: f"{x:.2f}")

print("\n" + "="*80)
print("REPORTE DE EJECUCIÓN MES 8 - ANÁLISIS DE GOBERNANZA")
print("="*80)
print(tabulate(df_final_table, headers="keys", tablefmt="psql", showindex=False))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')